## Project Name:  VacationPy

## Objectives:
* Ultilize the gmaps module with heatmap_layer feature to create humidity distribution (profile) across the globe. 
* Search for ideal climate cities, ultilize Google API's near-by search to locate hotels around those areas of interest.
* Ultilize gmaps module with add_layer of markers to show the hotel locations.

## Quick Observations:
* Climate conditions are the keys to search for ideal vacation locations. Logically, the ideal locations are in the "Temperate" climate types. 
* Out of 600+ cities being analyzed, only a few have hotels within 5,000 meters (3.1 miles). Majority is in Africa (could be an ideal continent for next summer vacation).

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import warnings
warnings.simplefilter("ignore")

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
read_path = os.path.join("output_data/cities.csv")
main_frame = pd.read_csv(read_path, low_memory=False)
main_frame.head()

,City_ID,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed
0,0,IN,1584214774,19.82,84.80,99,76,73.89,6.49
1,1,RU,1584214125,70.63,147.92,69,98,-28.14,4.25
2,2,AU,1584214737,-33.87,121.90,75,88,64.40,16.11
3,3,CK,1584214095,-21.21,-159.78,82,78,82.40,5.82
4,4,AU,1584214189,-33.65,115.33,83,98,66.00,13.27


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
type(main_frame['Humidity'][1])

numpy.int64

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# store locations
locations = main_frame[['Latitude', 'Longitude']]

# Fill NaN values and convert to float
humidity = main_frame["Humidity"].astype(float)

In [5]:
main_frame["Max Temp"].max()

104.0

In [6]:
# plot gmaps
figure = gmaps.figure() # this is to make a base gmap

# add humidity layer on top of gmap
humidity_layer = gmaps.heatmap_layer(locations,
                                     weights = humidity,  # == weights
                                     dissipating = True,
                                     max_intensity = 105,
                                     point_radius = 6
                                     )
figure.add_layer(humidity_layer)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# filter weather conditions:
maxtemp_df = main_frame.loc[(main_frame['Max Temp'] < 80) &
                             (main_frame['Max Temp'] > 70) &
                             (main_frame['Cloudiness'] == 0) &
                             (main_frame['Wind Speed'] < 10),
                              :]
maxtemp_df.dropna(how='any', inplace=True)


In [8]:
# quick dummy check to make sure the cloudiness in the entire main_frame is not 100% zero before being filtered into maxtemp_df
main_frame['Cloudiness'].describe()

count    576.000000
mean      54.163194
std       38.662298
min        0.000000
25%       20.000000
50%       63.500000
75%       92.000000
max      100.000000
Name: Cloudiness, dtype: float64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# store into hotel_df (= maxtemp_df)
hotel_df = maxtemp_df
hotel_df.set_index('City_ID', inplace=True)
hotel_df

,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed
City_ID,,,,,,,,
52,MR,1584214543,20.50,-10.07,0,7,71.69,4.61
173,NE,1584214651,18.74,7.39,0,11,75.11,8.34
193,BR,1584214305,-24.79,-50.01,0,64,72.05,1.92
255,IN,1584214813,16.78,80.30,0,86,72.41,4.27
298,CM,1584214819,12.08,15.03,0,11,78.80,5.82
307,YE,1584214821,13.17,44.58,0,69,73.17,7.56
425,MZ,1584214839,-24.53,32.98,0,76,77.13,7.45


In [10]:
hotel_list = []
city_list = []
for index, row in hotel_df.iterrows(): 
    targ_lat = row['Latitude']
    targ_long =  row['Longitude']
    target_coordinates = f'{targ_lat}, {targ_long}'
    target_radius = 5000 # search within 1 mile radius
    target_type = "lodging"

#   set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#     print(f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_coordinates}&radius={target_radius}&type={target_type}&key={g_key}')
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    if response['status'] == "ZERO_RESULTS":
        hotel_list.append('None')
        city_list.append('N/A')
    else:
        hotel_name = response["results"][0]["name"]
        hotel_list.append(hotel_name)
        city_country = response["results"][0]['plus_code']["compound_code"]
        
        # after extracted city and country mixed together
        # thus city names need to be extracted
        city_country_raw = city_country.split(', ')
        city_name = city_country_raw[-2].split()
        city_list.append(city_name[-1])
        
        
hotel_df['Hotel Name'] = hotel_list
hotel_df['City'] = city_list
hotel_df


,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name,City
City_ID,,,,,,,,,,
52,MR,1584214543,20.50,-10.07,0,7,71.69,4.61,None,N/A
173,NE,1584214651,18.74,7.39,0,11,75.11,8.34,Hôtel Telwa Bungalow,Arlit
193,BR,1584214305,-24.79,-50.01,0,64,72.05,1.92,CHACARA BAILLY,Paraná
255,IN,1584214813,16.78,80.30,0,86,72.41,4.27,Radisson,Pradesh
298,CM,1584214819,12.08,15.03,0,11,78.80,5.82,Résidence le Palmier,N'Djamena
307,YE,1584214821,13.17,44.58,0,69,73.17,7.56,None,N/A
425,MZ,1584214839,-24.53,32.98,0,76,77.13,7.45,Complexo Sonho Real,Chokwe


In [11]:
# If the value of hotel is none, data will be filtered out and thus not plot
valid_hotel_df = hotel_df[hotel_df['Hotel Name'] != 'None']
valid_hotel_df

,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name,City
City_ID,,,,,,,,,,
173,NE,1584214651,18.74,7.39,0,11,75.11,8.34,Hôtel Telwa Bungalow,Arlit
193,BR,1584214305,-24.79,-50.01,0,64,72.05,1.92,CHACARA BAILLY,Paraná
255,IN,1584214813,16.78,80.30,0,86,72.41,4.27,Radisson,Pradesh
298,CM,1584214819,12.08,15.03,0,11,78.80,5.82,Résidence le Palmier,N'Djamena
425,MZ,1584214839,-24.53,32.98,0,76,77.13,7.45,Complexo Sonho Real,Chokwe


In [12]:
# Using the template add the hotel marks to the heatmap
info_box_template = """"
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in valid_hotel_df.iterrows()]
marker_locations = valid_hotel_df[["Latitude", "Longitude"]]
marker_locations

,Latitude,Longitude
City_ID,,
173,18.74,7.39
193,-24.79,-50.01
255,16.78,80.30
298,12.08,15.03
425,-24.53,32.98


In [16]:
# Add marker layer ontop of heat map
# Create a marker layer using the poverty list to fill the info box

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content = hotel_info)

fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))